In [1]:
import sys
sys.path.insert(0, '/Users/timkrausz/Documents/GitHub/DA_maze_paper/DA/Modules/')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
plt.style.use('default')
plt.style.use("seaborn-poster")
from multi_rat_da import *
from hexLevelAnalyses import *
from photometryQuantifications import *
%matplotlib qt

# load dataframe and create object with relevant initial attributes

In [3]:
hexData = PhotRats(None)
# the directory containing your dataframe
loadpath = "/Volumes/Tim/Photometry/10MfRatDataSet/data4sharing/"
datName = "hexLevelDf"
hexData.df = reduce_mem_usage(pd.read_csv(loadpath+datName+".csv",index_col=[0]))
hexData.directory_prefix = loadpath

Memory usage of dataframe is 52.45 MB
Memory usage after optimization is: 12.54 MB
Decreased by 76.1%


### generate port-level q values if needed (requires parameters to have been optimized previously in julia)

In [ ]:
create_tridf(hexData)
hexData.triframe = hexData.tridf.copy()
del hexData.tridf
hexData.triframe.loc[:,"lrchoice"] = make_lr(hexData.triframe.port.values)
hexData.get_portQvals(qtype="port",level="rat")
hexData.bin_size = 1

# plot hex-level ramps under different conditions

In [140]:
fig = plot_hexRamp(hexData,ylim=[-.17,0.47])
fig.savefig(hexData.directory_prefix+"/hexRamp_mapDist.pdf")

In [16]:
fig = plot_hexRampByPrwd(hexData,stHex=14,ylim=[-.17,0.47])
fig.savefig(hexData.directory_prefix+"hexRampByPrwd2ndHalf_mapDist.pdf")

In [14]:
fig = plot_hexRampByPrwd(hexData,var='vel',stHex=14,ylim=[10,50])
fig.savefig(hexData.directory_prefix+"hexSpeedRampByPrwd2ndHalf_mapDist.pdf")

# compute effects of prior reward at chosen port, other port, and previous port, on DA in each hex during port-approach

In [21]:
create_tridf(hexData)

get_portHist(hexData)

get_back2samePortTrials(hexData)

add_rewardLagColumns2Df(hexData,lags=5)

add_pathSpecificRwdLagColumns2Df(hexData,lags=5)

add_pathSimilarity2df(hexData)

add_otherPortPriorRewardOutcome(hexData)

add_lastRewardColumn(hexData)

In [ ]:
fig = plotDaChosenVotherVlastRegInSpace(hexData)

# plot average of 0100 reward sequence

In [ ]:
rewardPattern = [0,1,0,0] # set this here for use throughout notebook

In [114]:
rPat = rewardPattern
entries = get_entriesByRwdSeq(hexData.df,rwd_pattern=rPat,stHex=10,alongPath=True,altPath=False,getPriorTri=False)
fig = plot_traceByRwdSeq(rPat,entries,stHex=10,fsize=[4.5,6])
plt.suptitle(str(np.shape(entries)[1])+" "+str(rPat)+" sequences")

Text(0.5, 0.98, '247 [0, 1, 0, 0] sequences')

In [115]:
fig.savefig(hexData.directory_prefix+"avgTraces_0100Pattern_samePath_mapDist.pdf")

# plot ramp increases and decreases from a single reward or omission

- plot result for each rat, where rat previously took any path to the reward port
- rpat specifies the reward pattern [reward outcome on t-2, reward outcome on t-1]

In [119]:
rPat = [0,1]
for rat in hexData.df.loc[:,"rat"].unique():
    entries = get_entriesByRwdSeq(hexData.df.loc[hexData.df.rat==rat,:],rwd_pattern=rPat,alongPath=False,getPriorTri=True)
    fig = plot_traceByRwdSeq(rPat,entries,fsize=[4.5,6],ylim=[None,None])
    plt.suptitle(str(np.shape(entries)[1])+" "+str(rPat)+" sequences for "+rat)
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.0)
    fig.savefig(hexData.directory_prefix+"/dataByRat/"+rat+"rampByRwdSeq_mapDist"+str(rPat)+".pdf")

# plot averages over rats under specified conditions
- if altPath, plot traces when rat previously took alternative path
- if alongPath, plot traces when rat previously took same path
- if both False, plot traces when rat previously took any path to the current port
- rpat specifies the reward pattern [reward outcome on t-2, reward outcome on t-1]

In [ ]:
plot_rampSequenceRatAvg(hexData,rPat=[0,1],altPath=False,alongPath=False)

## plot distance of peak reward effect

In [ ]:
plot_distOfPeakRwdEffect(hexData,hbin=2)

## Remove avg ramp, scale and subtract from individual-run DA, compute null residuals, and assess significance at each hex along the path

- calculate average ramps for scaling

In [ ]:
calc_hexRampRatDict(hexData,stHex=10)

- calculate residuals for same reward pattern as earlier plots

In [164]:
rpat = rewardPattern
rwd_pattern_inds,rat_IDs = getRwdPatternInds(hexData.df.copy(),rpat,alongPath=True,getPriorTri=True)
stHex=10
residuals = [[] for _ in range(len(rpat)+1)]
for patternedEntryInds,rat in zip(rwd_pattern_inds,rat_IDs):
    for e in range(len(residuals)):
        trace = hexData.df.loc[patternedEntryInds[e]-stHex:\
                                    patternedEntryInds[e],"DA"].values
        residuals[e].append(calcRampResiduals(hexData,rat,trace))

- perform 1000 shuffles of reward sequence from same data subset, and compute null residuals for each

In [196]:
nullResids = [get_NullResidualsByRwdSeq(hexData,[0,1,0,0],getPriorTri=True) for i in range(1000)]

In [204]:
fig = plotResidualsByRwdSeq(hexData,rpat,residuals,nullResids,fsize=[7,10],stHex=10,linwid=9)